**1. LLM, GPT, & Chat GPT API 사용하기**
- openAI key 생성 URL: https://platform.openai.com/api-keys


In [ ]:
!pip  install openai langchain unstructured pypdf pdf2image docx2txt pdfminer tiktoken sentence_transformers

In [40]:
import os
openai_api_key = 'Your KEY'
os.environ["OPENAI_API_KEY"]=openai_api_key

In [4]:
import langchain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


In [5]:
# GPT
gpt = OpenAI(model_name='gpt-3.5-turbo-instruct')
answer_gpt = gpt.predict("What is the most popular Large language model?")
print("answer_gpt", answer_gpt)



answer_gpt 

The most popular large language model is currently GPT-3 (Generative Pre-trained Transformer-3), developed by OpenAI.


In [10]:
# ChatGPT: 답변을 한국어로 받아보기
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
answer_chatgpt = chatgpt.predict("What is the most popular Large language model? answer in Korean")
print("chatGPT", answer_chatgpt)

chatGPT 가장 인기 있는 대형 언어 모델은 GPT-3입니다.


In [14]:
# Hyperparameter tuning
# chatgpt 답변 일관성 0: 일관적-default, 1: 같은 질문 받더라도 다르게 무작위성이 부여됨, 2: 가장 창의적)

chatgpt_t0 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
answer_t0 = chatgpt_t0.predict("Could you recommend good places we should go in Korea?")
print(answer_t0)

chatgpt_t1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)
answer_t1 = chatgpt_t1.predict("Could you recommend good places we should go in Korea?")
print(answer_t1)


Sure! Here are some popular places to visit in Korea:

1. Seoul: The capital city of Korea, Seoul is a bustling metropolis with a mix of modern skyscrapers and historic palaces. Don't miss visiting Gyeongbokgung Palace, Bukchon Hanok Village, and N Seoul Tower.

2. Jeju Island: Known for its stunning natural landscapes, Jeju Island is a popular destination for hiking, beaches, and exploring lava tubes and waterfalls.

3. Busan: Korea's second-largest city, Busan is famous for its beaches, seafood markets, and vibrant nightlife. Be sure to visit Haeundae Beach, Gamcheon Culture Village, and Jagalchi Fish Market.

4. Gyeongju: Often referred to as the "museum without walls," Gyeongju is a city filled with ancient temples, burial mounds, and historic sites. Don't miss visiting Bulguksa Temple, Seokguram Grotto, and the Royal Tombs.

5. Incheon: Located just outside of Seoul, Incheon is a port city known for its Chinatown, Songdo International Business District, and Incheon Grand Park.

Th

In [15]:
# 실시간으로 답변 받아보기: streaming=True
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=1)
chatgpt.predict("Could you recommend good places we should go in Korea?")


1. Seoul: Visit iconic landmarks such as Gyeongbokgung Palace, N Seoul Tower, and Bukchon Hanok Village. Don't miss out on exploring vibrant neighborhoods like Myeongdong and Insadong.

2. Jeju Island: Known for its stunning natural beauty, Jeju Island offers an array of attractions including Seongsan Ilchulbong Peak, Manjanggul Cave, and Jeju Loveland.

3. Busan: Explore the bustling city of Busan and enjoy the beautiful Haeundae Beach, Gamcheon Culture Village, and Beomeosa Temple.

4. Gyeongju: Immerse yourself in Korea's ancient history by visiting historic sites such as Bulguksa Temple, Seokguram Grotto, and the Royal Tombs in Gyeongju.

5. Incheon: Discover the vibrant port city of Incheon and visit attractions such as the Songdo Central Park, Incheon Grand Park, and Wolmido Island.

6. Andong: Experience traditional Korean culture in Andong by visiting Hahoe Folk Village, Dosan Seowon Confucian Academy, and the Andong Mask Dance Festival.

7. Pyeongchang: Enjoy outdoor activitie

"1. Seoul: Visit iconic landmarks such as Gyeongbokgung Palace, N Seoul Tower, and Bukchon Hanok Village. Don't miss out on exploring vibrant neighborhoods like Myeongdong and Insadong.\n\n2. Jeju Island: Known for its stunning natural beauty, Jeju Island offers an array of attractions including Seongsan Ilchulbong Peak, Manjanggul Cave, and Jeju Loveland.\n\n3. Busan: Explore the bustling city of Busan and enjoy the beautiful Haeundae Beach, Gamcheon Culture Village, and Beomeosa Temple.\n\n4. Gyeongju: Immerse yourself in Korea's ancient history by visiting historic sites such as Bulguksa Temple, Seokguram Grotto, and the Royal Tombs in Gyeongju.\n\n5. Incheon: Discover the vibrant port city of Incheon and visit attractions such as the Songdo Central Park, Incheon Grand Park, and Wolmido Island.\n\n6. Andong: Experience traditional Korean culture in Andong by visiting Hahoe Folk Village, Dosan Seowon Confucian Academy, and the Andong Mask Dance Festival.\n\n7. Pyeongchang: Enjoy outd

In [17]:
# ChatGPT에 역할 부여하기
from langchain.schema import AIMessage, HumanMessage, SystemMessage
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
message = [
    # ChatGPT 역할을 부여
    SystemMessage(
        content="You are a world tour guide. Please recommend a good place from where I mentioned it"
    ),
    # 이 문장에 대해 역할대로 수행
    HumanMessage(
        content="I want to go France"
    )
]
response_langchain = chatgpt(message)
response_langchain.content

'I recommend visiting the beautiful city of Paris in France. Known as the "City of Light," Paris is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Champs-Élysées. You can also explore charming neighborhoods like Montmartre, enjoy delicious French cuisine at local cafes, and experience the vibrant art and culture scene. Don\'t forget to take a leisurely stroll along the Seine River and immerse yourself in the romantic atmosphere of this enchanting city.'

**2. Prompt 실습**
- model에 대한 입력을 구성, LangChain은 이 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 함수를 제공

In [21]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 일반 LLM
string_prompt = PromptTemplate.from_template("good places in {subject}") # 프롬프트 템플릿을 통해 매개변수로 삽입 가능한 문자열로 변환
string_prompt_value = string_prompt.format_prompt(subject="seoul")
print("string_prompt_value", string_prompt_value)
# Chat LLM
chat_prompt = ChatPromptTemplate.from_template("good places in {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject="seoul")
print("chat_prompt_value", chat_prompt_value)

string_prompt_value text='good places in seoul'
chat_prompt_value messages=[HumanMessage(content='good places in seoul')]


**GPT와 PromptTemplate**

In [22]:
from langchain.prompts.prompt import PromptTemplate
template = """

You are a world tour guide. Please recommend a good place from where I mentioned it

<places>
{places}

"""
prompt_template = PromptTemplate(
    input_variables=['places'],
    template = template
)
# 여러개 질문 가능한 템플릿
print(prompt_template.format(places='korea, paris, new york, london'))
print(gpt(prompt_template.format(places='korea, paris, new york, london')))



You are a world tour guide. Please recommend a good place from where I mentioned it

<places>
korea, paris, new york, london




**ChatGPT와 ChatPromptTemplate**


In [25]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

# chatGPT 모델
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
# chatGPT 역할
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
# 사용자 입력 
human_template = "{places}"
human_message_promt = HumanMessagePromptTemplate.from_template(human_template)


chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_promt])
answer = chatgpt(chat_prompt.format_prompt(places='korea, paris, new york, london').to_messages())
print(answer.content)


I would recommend visiting Paris, France. Paris is known for its beautiful architecture, world-class museums like the Louvre, iconic landmarks such as the Eiffel Tower, and delicious cuisine. Stroll along the Seine River, explore charming neighborhoods like Montmartre, and immerse yourself in the rich culture and history of this romantic city. Don't forget to indulge in some pastries and coffee at a cozy café while people-watching!


**3. Document Loaders 실습하기**

- web 에 있는 Text 불러오기
- pdf 에 있는 Text 불러오기

In [26]:
from langchain.document_loaders import WebBaseLoader
# WebPage
loader = WebBaseLoader("https://edition.cnn.com/2024/03/16/politics/biden-trump-january-6/index.html")
data = loader.load()
print(data[0].page_content)

 








Why does Biden keep mentioning January 6? Because Trump won‚Äôt stop talking about it | CNN Politics






































































































CNN values your feedback




                                                        1. How relevant is this ad to you?
                                                






























                                                2. Did you encounter any technical issues?
                                        











                                                                        Video player was slow to load content
                                                                        



                                                                        Video content never loaded
                                                                        



                                                                        Ad froze or 

In [27]:
from langchain.document_loaders import PyPDFLoader
# pdf
loader = PyPDFLoader("paper-1-SCL.pdf")
pages = loader.load_and_split()

**4. Text Splitter 실습하기**
- 텍스트를 Chunk 단위로 분할
- CharacterTextSplitter: 특정 구분자를 기준으로 텍스트를 정크로 분할, 단, Charactor Split은 Chunf size를 초과하는 경우가 발생하여 대부분 Recursive 모듈을 자주사용
- RecursiveCharacterTextSplitter: 일반적인 글로 된 문서는 모두 textsplitter로 분할

In [32]:
from langchain.text_splitter import CharacterTextSplitter
with open("paper-1-SCL_abstract_intro_relatedwork.txt") as f:
    state_of_the_union = f.read()

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
)
texts = text_splitter.split_text(state_of_the_union)
print(texts[0])
print("-"*100)
print(texts[1])
print("-"*100)
print(texts[2])

Created a chunk of size 1362, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 2019, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 1212, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000


Abstract—Classification of multisensor signals is an im- portant problem in maintaining stable process operations, particularly in advancing predictive modeling for early de- tection of abnormal states. Self-supervised learning meth- ods, one of the representation learnings, have been widely studied. However, they have focused on using unlabeled data. In this study, we aim to address the challenge of effectively utilizing fully labeled data for modeling multi- sensor signals. We introduce supervised contrastive learn- ing (SCL) for the classification of multisensor signals. Our training framework involves a two-step process: SCL for en- coder pretraining with time-series data augmentations, and classifier training with the pretrained encoder. Our method exhibits superior performance, outperforming traditional supervised learning approaches by a substantial margin. Furthermore, we demonstrate the practical applicability of our approach for early prediction problems through experi- ments

In [33]:
print(pages[1].page_content)

including fault detection and diagnosis and machinery health
monitoring [4],[5],[6],[7]. Nevertheless, they make an essential
demand for manual feature extraction or dimensionality reduc-
tion steps [3]. It is difﬁcult to perform the joint optimization of
the feature extraction method and the machine learning model,resulting in the inability of the extracted features to directly
represent the abnormal status of equipment [8]. Consequently,
exploring an effective method for representation learning ofmultisensor signal data becomes imperative.
Deep learning models incorporate representation learning and
classiﬁcation tasks in uniﬁed network architecture. Representa-
tion learning involves training models to automatically discover
and create meaningful representations or features from raw data.Long short-term memory and convolutional neural networks
(CNNs) have become the primary methods with their outstand-
ing predictive performance in time-series modeling [1],[9],[10].
Although the CNN

In [35]:
texts = text_splitter.split_documents(pages)
print(texts[1].page_content)

including fault detection and diagnosis and machinery health
monitoring [4],[5],[6],[7]. Nevertheless, they make an essential
demand for manual feature extraction or dimensionality reduc-
tion steps [3]. It is difﬁcult to perform the joint optimization of
the feature extraction method and the machine learning model,resulting in the inability of the extracted features to directly
represent the abnormal status of equipment [8]. Consequently,
exploring an effective method for representation learning ofmultisensor signal data becomes imperative.
Deep learning models incorporate representation learning and
classiﬁcation tasks in uniﬁed network architecture. Representa-
tion learning involves training models to automatically discover
and create meaningful representations or features from raw data.Long short-term memory and convolutional neural networks
(CNNs) have become the primary methods with their outstand-
ing predictive performance in time-series modeling [1],[9],[10].
Although the CNN

In [37]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
tokenizer = tiktoken.get_encoding("cl100k_base")
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)


loader = PyPDFLoader("paper-2-QDL.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

In [38]:
texts

[Document(page_content='4374 IEEE TRANSACTIONS ON SYSTEMS, MAN, AND CYBERNETICS: SYSTEMS, VOL. 52, NO. 7, JULY 2022\nQuality-Discriminative Localization of Multisensor\nSignals for Root Cause Analysis\nYoon Sang Cho and Seoung Bum Kim\nAbstract —Root cause analysis (RCA) methods for effectively\nidentifying the critical causes of abnormal processes haveattracted attention because manufacturing processes have grown\nin scale and complexity. However, the existing methods for\nbuilding automatic RCA models suffer from the disadvantageof typically requiring expert knowledge. In addition, withouta dataset representing the causal relationship of multivari-ate processes, it is difﬁcult to provide useful information forRCA. Although data-driven RCA methods have been proposed,most are based on classiﬁcation models. Given that productquality is deﬁned as a continuous variable in many manufac-turing industries, classiﬁcation models are limited in derivingroot causes affecting the product quality 

**5. Text Embedding 실습하기**

- 텍스트 문서를 수치 변환하고 질문과 유사도를 비교한다

In [42]:
from langchain.embeddings import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

embeddings = embeddings_model.embed_documents(
    [
        "안녕하세요",
        "제 이름은 조윤상입니다.",
        "조윤상은 산업경영공학을 전공했습니다",
        "언어모델은 대용량 텍스트데이터를 학습한 생성형 딥러닝 모델입니다",
        "Hello World!",
        
    ]
)
len(embeddings), len(embeddings[0])
embedded_query_q = embeddings_model.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = embeddings_model.embed_query("이 대화에서 언급된 이름은 조윤상입니다.")
print(len(embedded_query_q), len(embedded_query_a))


1536 1536


In [44]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, embeddings[0]))
print(cos_sim(embedded_query_q, embeddings[1]))
print(cos_sim(embedded_query_q, embeddings[2]))
print(cos_sim(embedded_query_q, embeddings[3]))
print(cos_sim(embedded_query_q, embeddings[4]))


0.8899458597020108
0.8158938956610856
0.8315212592497677
0.7884215414829497
0.8002830634820086
0.7278030214139422


In [59]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "jhgan/ko-sbert-nli" # korean
model_name = "BAAI/bge-small-en" # englilsh
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs,
)
documents = [
    [
        "my name is YoonSangCho",
        "I majored in Industrial & Management Engineering",
        "AI is powerful",
        "where are you from?",
        "Hello World",
    ]
]
embeddings = hf.embed_documents(
    [
        "my name is YoonSangCho",
        "I majored in Industrial & Management Engineering",
        "AI is powerful",
        "where are you from?",
        "Hello World",
    ]
)

BGE_query_q = hf.embed_query("Hello? who is this?")
BGE_query_a = hf.embed_query("Hi this is YoonSangCho")

print(cos_sim(BGE_query_q, BGE_query_a))
print(cos_sim(BGE_query_q, embeddings[0]))
print(cos_sim(BGE_query_q, embeddings[1]))
print(cos_sim(BGE_query_q, embeddings[2]))
print(cos_sim(BGE_query_q, embeddings[3]))
print(cos_sim(BGE_query_q, embeddings[4]))


0.8494187267618187
0.8037215308622446
0.7287845465894771
0.7491667804945571
0.7872170923464111
0.7816865711989947


In [60]:
BGE_query_q

[-0.05165728181600571,
 -0.014959916472434998,
 0.017631476745009422,
 -0.041672028601169586,
 0.034519705921411514,
 -0.0357469841837883,
 0.08076842129230499,
 0.02722739614546299,
 0.02002798207104206,
 0.010667991824448109,
 0.032536379992961884,
 -0.06471751630306244,
 -0.008188118226826191,
 0.03445412963628769,
 0.007697741966694593,
 -0.029890544712543488,
 0.051178399473428726,
 -0.016601573675870895,
 -0.010365455411374569,
 0.013720612972974777,
 0.008365671150386333,
 0.030398201197385788,
 -0.058151889592409134,
 -0.049041878432035446,
 0.025917071849107742,
 -0.010762125253677368,
 -0.004442320205271244,
 0.005020192824304104,
 0.006352833937853575,
 -0.1459188163280487,
 -0.024220235645771027,
 -0.02214004099369049,
 0.013377167284488678,
 -0.0013176710344851017,
 0.005801036953926086,
 -0.027996528893709183,
 0.007844232022762299,
 -0.0014156651450321078,
 -0.04077493026852608,
 -0.011248945258557796,
 0.0103797297924757,
 -0.007679881062358618,
 0.01887989602982998,
 -

In [66]:
list_sim = [cos_sim(BGE_query_q, embeddings[c]) for c in range(len(embeddings))]
idx_max_sim = np.where(list_sim==max(list_sim))[0][0]
print(idx_max_sim)
print(documents[0][idx_max_sim])


0
my name is YoonSangCho


In [65]:
idx_max_sim
documents

[['my name is YoonSangCho',
  'I majored in Industrial & Management Engineering',
  'AI is powerful',
  'where are you from?',
  'Hello World']]

embeddings

**6. VectorStore**
- embedding 처리 후 이들을 저장하는 벡터 저장소로 대표적 모듈로는 아래와 같다
- (1) Chroma
- (2) FAISS

In [ ]:
!pip isntall chromadb tiktoken transformers sentence_trannsformers faiss-cpu

In [46]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("paper-0-multimodal.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

# create the open_source embedding function
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli" # korean
model_name = "BAAI/bge-small-en" # englilsh
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

# 임베딩 벡터를 Chroma에 넣기
db = Chroma.from_documents(docs, hf)

# Query 
query  = "What is the multomodal foundation models?"
docs = db.similarity_search(query)

print(docs[0].page_content)


Multimodal
Foundation
ModelsSpecific-Purpose
Pre-trained ModelsVisual
Understanding §2Supervised LearningBiT (Kolesnikov et al., 2020);
ViT (Dosovitskiy et al., 2021)
Contrastive Language-
Image Pre-trainingCLIP (Radford et al., 2021);
ALIGN (Jia et al., 2021)
Image-only Self-supervised LearningMoCo (He et al., 2020); DINO (Caron
et al., 2021); MAE (He et al., 2022a)
Synergy Among Different MethodsSLIP (Mu et al., 2021);
UniCL (Yang et al., 2022b)
Multimodal FusionUNITER (Chen et al., 2020d);
CoCa (Yu et al., 2022a)
Region-level and Pixel-
level Pre-trainingGLIP (Li et al., 2022e);
SAM (Kirillov et al., 2023)
Visual
Generation §3Overview: Text-to-Image GenerationStable Diffusion (Rom-
bach et al., 2021)
Spatial Controllable GenerationControlNet (Zhang
and Agrawala, 2023)
Text-based Editing InstructPix2Pix (Brooks et al., 2023)
Text Prompts Following DDPO (Black et al., 2023)
Concept Customization DreamBooth (Ruiz et al., 2023)
General-Purpose
AssistantsUnified Vision
Models §4From Clos

In [47]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

# (1) document loader
loader = PyPDFLoader("paper-1-SCL.pdf")
pages = loader.load_and_split()

# (2)  Text Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function = tiktoken_len)
docs = text_splitter.split_documents(pages)

# (3) embedding
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf_scl = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)
# (4) Vector Store
db = FAISS.from_documents(docs, hf_scl)
query = "what is the supervised contrastive learning?"
docs = db.similarity_search(query) # 유사도
print(docs[0].page_content)

augmentations is required to guarantee predictive performancebecause, otherwise, abnormal symptoms may not become no-
ticeable early enough to prevent an incident. It also necessitates
data augmentations to enforce models to learn various scalesand patterns of abnormal signals.
However, most existing approaches have largely focused
on manual feature extraction methods of multisensor signals,revealing limitations for real-world applicability. Speciﬁcally,
the traditional supervised learning approaches have overlooked
data augmentation designed for time-series data in the contextof multisensor signals in manufacturing contexts. While recent
studies have made strides in leveraging self-supervised learn-
ing for representation learning, they have focused on utilizingunlabeled data, neglecting the potential beneﬁts of fully labeled
data.
Our study addresses these limitations by introducing super-
vised contrastive learning (SCL) that leverages data augmenta-
tions and contrastive learning t

**6. Retrieval을 통해 LLM에게 물어보기**

In [48]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

# document loader
loader = PyPDFLoader("paper-2-QDL.pdf")
pages = loader.load_and_split()

# text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

# embeddiing
from langchain.embeddings import HuggingFaceEmbeddings
# model_name = "all-mpnet-base-v2"
# model_name = "jhgan/ko-sbert-nli"
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs,
)
# VectorStore
docsearch = Chroma.from_documents(texts, hf)

# LLM
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

openai = ChatOpenAI(model_name = "gpt-3.5-turbo",
                    streaming=True,
                    callbacks=[StreamingStdOutCallbackHandler()],
                    temperature = 0
                    )

qa = RetrievalQA.from_chain_type(llm=openai,
                                 chain_type="stuff", ####
                                 retriever = docsearch.as_retriever(
                                     # chroma 객체를 벡터 저장소를 쓰는게 아니라 검색기로 쓰겠다
                                     search_type="mmr", # 연관성 높은 문서를 뽑아올 때 최대한 다양한 소스를 바탕으로 답변을 생성하도록 설정
                                     search_kwargs={'k':3, 'fetch_k': 10}),# mmr 관련 변수
                                 return_source_documents = True
                                 )

query = "explain how to analyze the root cause of fault symtoms using multisensor signal data"
result = qa(query)

/Users/yoonsangcho/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


To analyze the root cause of fault symptoms using multisensor signal data, you can follow these steps based on the information provided in the context:

1. **Data Collection**: Utilize advanced sensor technology to collect real-time multisensor signal data related to the process or system you are analyzing. Ensure that the data includes information on sequential processes, processing times, and product quality.

2. **Abnormal Signal Detection**: Detect abnormal signals within the collected sensor data. Abnormal signals are directly associated with abnormal process states and decreased product quality. These abnormal signals are crucial indicators of potential faults or issues.

3. **Multilevel Cause Identification**: Identify multilevel causes by analyzing the sensor data. Problems often occur due to a combination of factors across different levels of the process. Understanding these multivariate processes and processing times is essential for identifying root causes accurately.

4. **

# 실습 해보기

In [ ]:
import os
os.environ["OPENAI_API_KEY"]= None # 본인 OPENAI_API_KEY를 입력하세요
import langchain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage


""" 
**1. ChatGPT 모델을 불러오고 사용자 선택변수를 조정하여 답변을 비교해보세요.**
- SystemMessage, HumanMessage를 변경하여 ChatGPT 역할을 부여하고 원하는 형식의 답변을 유도해보세요.
- 사용자 선택변수 model_name, temperature을 조정하여 답변 차이를 비교해보세요.
"""
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
message = [
    SystemMessage(
        content=None # ChatGPT에게 역할을 부여하세요
    ),
    HumanMessage(
        content=None # ChatGPT에게 물어보고 싶은 내용을 입력하세요.
    )
]
answer = None # chatgpt와 지정한 message를 이용하여 답변을 출력하세요. 이때, chatgpt가 답변한 내용(content) 만 출력하세요.


In [49]:

""" 
**2. ChatGPT 모델을 불러오고 원하는 PDF 데이터를 입력하여 정보를 검색할 수 있도록 ChatGPT를 구축하세요.**
- 본인 현업에서 가장 적용하고 싶은 AI기술을 다룬 논문을 찾아 로드하세요.
- ChatOpenAI 모듈에서 model_name 조정하여 답변 차이를 비교해보세요.
- RetrievalQA.from_chain_type 모듈에서 chain_type 조정하여 답변 차이를 비교해보세요.
"""

from langchain.document_loaders import PyPDFLoader
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# (1) Document loaders
loader = None # 본인이 원하는 PDF를 로드하세요.
pages = loader.load_and_split()

# (2) Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

# (3) Embeddinng
model_name = None # 본인이 원하는 EMBEDDING model_name 지정하세요.
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs,
)

# (4) VectorStores
docsearch = Chroma.from_documents(texts, hf)

# (5) LLM
openai = ChatOpenAI(model_name = None # 본인이 원하는 LLM model_name 지정하세요.
                    streaming=True,
                    callbacks=[StreamingStdOutCallbackHandler()],
                    temperature = 0
                    )

# (6) RetrievalQA
qa = RetrievalQA.from_chain_type(llm=openai,
                                 chain_type= None, # chain_type을 지정하세요
                                 retriever = None, # retriever 지정하세요
                                 return_source_documents = None, # return_source_documents 지정하세요
                                 )

query = None # 본인 직무에 적용할 수 있는 방법론과 관련된 질문을 ChatGPT에게 물어보세요.
result = qa(query)

SyntaxError: invalid syntax (222686886.py, line 47)